# Import Library

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import keras
import random
import math
import time
from cv2 import cv2
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, AveragePooling2D, BatchNormalization, Activation, MaxPooling2D, MaxPool2D
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras import regularizers, optimizers
from google.colab import drive
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils.np_utils import to_categorical
from keras.optimizers import RMSprop, Adam
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg19 import VGG19
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, classification_report

# Helper Functions

In [ ]:
# Result Print Function
def result_print (model):
  scoreTrain = model.evaluate(train_images, y_train, verbose=1)
  print("Train loss: {} - Train accuracy: {}".format(scoreTrain[0], scoreTrain[1]))
  print()
  scoreVal = model.evaluate(valid_images, y_val, verbose=1)
  print("Validation loss: {} - Validation accuracy: {}".format(scoreVal[0],scoreVal[1]))
  print()
  scoreTest = model.evaluate(test_images, y_test, verbose=1)
  print("Test loss: {} - Test accuracy: {}".format(scoreTest[0],scoreTest[1]))
  return 

In [ ]:
#Print Metrics
def print_metrics(model):
  ypred  = model_VGG19.predict(test_images)
  ypred_bool = np.argmax(y_pred, axis=1)

  rounded_labels=np.argmax(y_test, axis=1)

  precision = precision_score(rounded_labels, ypred_bool, average="macro")
  recall = recall_score(rounded_labels, y_pred, average="macro")
  f1 = f1_score(rounded_labels, y_pred, average="macro")

  print("Precision: " + str(round(precision, 6)))
  print("Recall: " + str(round(recall, 6)))
  print("f1: " + str(round(f1, 6)))
return rounded_labels, ypred_bool

In [ ]:
#Confusion Matrix
def confusion_matrix(rounded_labels,ypred_bool):
  disp = confusion_matrix(y_true=rounded_labels, y_pred=ypred_bool)
  f,ax = plt.subplots(figsize=(8, 8))
  sns.heatmap(disp, annot=True, linewidths=0.01,cmap="Greens",linecolor="gray", fmt= '.1f',ax=ax)
  plt.xlabel("Predicted Label")
  plt.ylabel("True Label")
  plt.title("Confusion Matrix")
  plt.show()

In [ ]:
# Plot Accuracy Function
def plot (history_model):
  plt.plot(history_model.history['accuracy'], label='train')
  plt.plot(history_model.history['val_accuracy'], label='test')
  plt.title("Accuracy Changes")
  plt.xlabel("Epochs")
  plt.ylabel("Acc")
  plt.legend()
  plt.show()

# Read Dataset with RGB

In [ ]:
class_names = [
    'correlation_strong_negative',
    'correlation_strong_positive',
    'correlation_weak_negative',
    'correlation_weak_positive',
    'non_correlation']

train_images = []
train_labels = []
test_images = []
test_labels = []
valid_images = []
valid_labels = []
train_count=600
test_count=200
valid_count=200
read_mode=cv2.IMREAD_GRAYSCALE
size = (160, 160)
interpolation_mode = cv2.INTER_AREA
train_path = '/content/gdrive/MyDrive/dataset_son/train/'
test_path = '/content/gdrive/MyDrive/dataset_son/test/'
valid_path = '/content/gdrive/MyDrive/dataset_son/validation/'


def grayscale_dataset(path1,data_count,data_images,data_labels):
  for i in range(1, data_count+ 1):
    for class_name in class_names:
      drive.mount('/content/gdrive')
      path = path1 + class_name + '/'+'Image (' + str(i) +')'+ '.png'
      print(path)
      img = cv2.imread(path)
      img = cv2.resize(img, dsize=size)
      img = img.reshape((size[0],size[1], 3))
      data_images.append(np.array(img))
      data_labels.append(np.array(class_names.index(class_name)))
  data_images = np.array(data_images)
  data_labels = np.array(data_labels)
  data_images = data_images / 255.0
  return data_images, data_labels
train_images, train_labels = grayscale_dataset(train_path,train_count,train_images,train_labels)
test_images, test_labels = grayscale_dataset(test_path,test_count,test_images,test_labels)
valid_images, valid_labels = grayscale_dataset(valid_path,valid_count,valid_images,valid_labels)


#Data Encoding

In [ ]:
#Encoding
y_train=to_categorical(train_labels)
y_val=to_categorical(valid_labels)
y_test=to_categorical(test_labels)

# Visualization Dataset

In [ ]:
data_to_show = x_train
labels_to_show = y_train

x = 19
index = random.randint(0, len(data_to_show) - x)

plt.figure(figsize=(20, 20))
j = 0

for i in range(index, index + x):
    plt.subplot(5, 5, j + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(data_to_show[i], cmap=plt.cm.binary)
    #plt.xlabel(str(math.floor((i / 4) + 1)) + " " + class_names[labels_to_show[i]])
    j += 1

plt.show()

# Augmentation

In [ ]:
datagen = ImageDataGenerator(
         shear_range = 8,
         zoom_range = 0.1,
         horizontal_flip = False,
         featurewise_center=False,
         samplewise_center=False,
         featurewise_std_normalization=False,
         samplewise_std_normalization=False,
         zca_whitening=False,
         height_shift_range=0.08, 
         brightness_range = (0.5, 0.5), 
         vertical_flip=False,
)
datagen.fit(train_images)

# VGG19 With Transfer Learning

In [ ]:
vgg19 = VGG19(include_top = False, weights="imagenet", input_shape=(160,160,3))

## Add Layer

In [ ]:
vgg19_layer_list = vgg19.layers
model_VGG19 = Sequential()
model_VGG19.add(vgg19)

## Transfer Learning False/True

In [ ]:
for layer in model_VGG19.layers:
  layer.trainable=False

## Add Neural Network Layers

In [ ]:
model_VGG19.add(Flatten())
model_VGG19.add(Dense(512,activation='relu'))
model_VGG19.add(Dense(5,activation='softmax'))

## VGG19 Compile and Fit

In [ ]:
batch_size = 128
epochs = 16
model_VGG19.compile(loss='categorical_crossentropy',optimizer=Adam(),metrics=['accuracy'])
startVGG19 = time.time()
history_VGG19=model_VGG19.fit_generator(datagen.flow(train_images,y_train, batch_size=batch_size), epochs=epochs,validation_data=(valid_images,y_val))
stopVGG19 = time.time()

In [ ]:
print(f"VGG19 Model with RGB Training time: {stopVGG19 - startVGG19}s")

## Print Metrics and Plot Accuracy

In [ ]:
# Result Print
result_print(model_VGG19)

In [ ]:
# Plot Accuracy
plot(history_VGG19)

In [ ]:
#Print Recall, Precision, F1-Score
rounded_labels, ypred_bool = print_metrics(model_VGG19)

In [ ]:
#Confusion Matrix
confusion_matrix(rounded_labels, ypred_bool)

# AlexNet Without Transfer Learning

## Create AlexNet

In [ ]:
#https://analyticsindiamag.com/hands-on-guide-to-implementing-alexnet-with-keras-for-multi-class-image-classification/
#Defining the parameters
batch_size= 128
epochs=13
learn_rate=.001

np.random.seed(1000)
#Instantiation
AlexNet = Sequential()

#1st Convolutional Layer
AlexNet.add(Conv2D(filters=96, input_shape=(160,160,3), kernel_size=(11,11), strides=(4,4), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#2nd Convolutional Layer
AlexNet.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#3rd Convolutional Layer
AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))

#4th Convolutional Layer
AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))

#5th Convolutional Layer
AlexNet.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#Passing it to a Fully Connected layer
AlexNet.add(Flatten())
# 1st Fully Connected Layer
AlexNet.add(Dense(4096))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
# Add Dropout to prevent overfitting
AlexNet.add(Dropout(0.4))

#2nd Fully Connected Layer
AlexNet.add(Dense(4096))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
#Add Dropout
AlexNet.add(Dropout(0.4))

#3rd Fully Connected Layer
AlexNet.add(Dense(1000))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
#Add Dropout
AlexNet.add(Dropout(0.4))

#Output Layer
AlexNet.add(Dense(5))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('softmax'))

#Model Summary
AlexNet.summary()

# Compiling the model
AlexNet.compile(loss = keras.losses.categorical_crossentropy, optimizer= 'adam', metrics=['accuracy'])

#Learning Rate Annealer
from keras.callbacks import ReduceLROnPlateau
lrr= ReduceLROnPlateau(monitor='val_accuracy',   factor=.01,   patience=3,  min_lr=1e-5) 

#Training the model
startAlexNet = time.time()

history_AlexNet=AlexNet.fit_generator(datagen.flow(train_images,y_train, batch_size=batch_size), epochs=epochs,validation_data=(valid_images,y_val))

stopAlexNet = time.time()

In [ ]:
print(f"AlexNet Model with RGB Training time: {stopAlexNet - startAlexNet}s")

## Result Print and Plot Accuracy


In [ ]:
result_print(AlexNet)

In [ ]:
# Plot Accuracy
plot(history_AlexNet)

In [ ]:
#Print Recall, Precision, F1-Score
rounded_labels, ypred_bool = print_metrics(AlexNet)

In [ ]:
#Confusion Matrix
confusion_matrix(rounded_labels, ypred_bool)

# Created Model


In [ ]:
batch_size= 128
epochs=16
num_classes =5

model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (3,3),padding='Same',activation ='relu', input_shape=(160,160,3)))
model.add(BatchNormalization())
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding='Same',activation ='relu'))
model.add(BatchNormalization())
model.add(AveragePooling2D(pool_size=(2,2)))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding='Same',activation ='relu'))
model.add(BatchNormalization())
model.add(AveragePooling2D(pool_size=(2,2)))
model.add(Dropout(0.1))
model.add(Conv2D(filters = 128, kernel_size = (3,3), activation ='relu',padding='Same'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 256, kernel_size = (3,3), activation ='relu', padding='Same'))
model.add(BatchNormalization())
model.add(AveragePooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes,activation='softmax'))
model.summary()

## Model Compile and Fit

In [ ]:
model.compile(loss = keras.losses.categorical_crossentropy,
              optimizer = Adam(),
              metrics=["accuracy"])
startCreatedModel = time.time()
history_model=model.fit_generator(datagen.flow(train_images,y_train, batch_size=batch_size), epochs=epochs,validation_data=(valid_images,y_val))
stopCreatedModel = time.time()

In [ ]:
print(f"Created Model with Grayscale Training time: {stopCreatedModel - startCreatedModel}s")

Created Model with Grayscale Training time: 247.2065851688385s


## Result Print and Plot Accuracy

In [ ]:
result_print(model)

In [ ]:
# Plot Accuracy
plot(history_model)

In [ ]:
#Print Recall, Precision, F1-Score
rounded_labels, ypred_bool = print_metrics(model)

In [ ]:
#Confusion Matrix
confusion_matrix(rounded_labels, ypred_bool)

# Model Learn Open CV

In [ ]:
#https://learnopencv.com/image-classification-using-convolutional-neural-networks-in-keras/

model_lo_cv = Sequential()
model_lo_cv.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(160,160,1)))
model_lo_cv.add(Conv2D(32, (3, 3), activation='relu'))
model_lo_cv.add(MaxPooling2D(pool_size=(2, 2)))
model_lo_cv.add(Dropout(0.25))
model_lo_cv.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model_lo_cv.add(Conv2D(64, (3, 3), activation='relu'))
model_lo_cv.add(MaxPooling2D(pool_size=(2, 2)))
model_lo_cv.add(Dropout(0.25))
model_lo_cv.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model_lo_cv.add(Conv2D(64, (3, 3), activation='relu'))
model_lo_cv.add(MaxPooling2D(pool_size=(2, 2)))
model_lo_cv.add(Dropout(0.25))
model_lo_cv.add(Flatten())
model_lo_cv.add(Dense(512, activation='relu'))
model_lo_cv.add(Dropout(0.5))
model_lo_cv.add(Dense(5, activation='softmax'))

### Model Learn Open CV Compile and Fit

In [ ]:
batch_size = 128
epochs = 24

model_lo_cv.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
startLearnOpenCV = time.time()
history_model_lo_cv=model_lo_cv.fit_generator(datagen.flow(train_images, y_train, batch_size=batch_size),epochs=epochs,validation_data=(valid_images, y_val))
stopLearnOpenCV = time.time()

In [ ]:
print(f"Learn Open CV Model with RGB Training time: {stopLearnOpenCV - startLearnOpenCV}s")

## Result Print and Plot Accuracy

In [ ]:
result_print(model_lo_cv)

In [ ]:
# Plot Accuracy
plot(history_model_lo_cv)

In [ ]:
#Print Recall, Precision, F1-Score
rounded_labels, ypred_bool = print_metrics(model_lo_cv)

In [ ]:
#Confusion Matrix
confusion_matrix(rounded_labels, ypred_bool)